In [1]:
%pylab inline
import re
import csv
import math
import string
from difflib import SequenceMatcher
from collections import Counter
from __future__ import division

Populating the interactive namespace from numpy and matplotlib


In [2]:
TEXT = open('big.txt').read() #read all the words from big.txt
len(TEXT)

6488666

In [3]:
def tokens(text):
    return re.findall('[a-z]+', text.lower()) 

In [4]:
tokens('hi there randy here*2')

['hi', 'there', 'randy', 'here']

In [5]:
WORDS = tokens(TEXT)
len(WORDS)

1105285

In [6]:
COUNTS = Counter(WORDS)

print(COUNTS.most_common(10))

[('the', 80030), ('of', 40025), ('and', 38313), ('to', 28766), ('in', 22050), ('a', 21155), ('that', 12512), ('he', 12401), ('was', 11410), ('it', 10681)]


In [7]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

### Spell Checker
        


In [8]:
def correct(word):
    candidates = (known(edits0(word)) or 
                  known(edits1(word)) or 
                  known(edits2(word)) or 
                  [word])
    return max(candidates, key=COUNTS.get)

Now for `edits1(word)`: the set of candidate words that are one edit away. For example, given `"wird"`, this would include `"weird"` (inserting an `e`) and `"word"` (replacing a `i` with a `o`), and also `"iwrd"` (transposing `w` and `i`; then `known` can be used to filter this out of the set of final candidates). How could we get them?  One way is to *split* the original word in all possible places, each split forming a *pair* of words, `(a, b)`, before and after the place, and at each place, either delete, transpose, replace, or insert a letter:

<table>
  <tr><td> pairs: <td><tt> Ø+wird <td><tt> w+ird <td><tt> wi+rd <td><tt>wir+d<td><tt>wird+Ø<td><i>Notes:</i><tt> (<i>a</i>, <i>b</i>)</tt> pair</i>
  <tr><td> deletions: <td><tt>Ø+ird<td><tt> w+rd<td><tt> wi+d<td><tt> wir+Ø<td><td><i>Delete first char of b</i>
  <tr><td> transpositions: <td><tt>Ø+iwrd<td><tt> w+rid<td><tt> wi+dr</tt><td><td><td><i>Swap first two chars of b
  <tr><td> replacements: <td><tt>Ø+?ird<td><tt> w+?rd<td><tt> wi+?d<td><tt> wir+?</tt><td><td><i>Replace char at start of b
  <tr><td> insertions: <td><tt>Ø+?+wird<td><tt> w+?+ird<td><tt> wi+?+rd<td><tt> wir+?+d<td><tt> wird+?+Ø</tt><td><i>Insert char between a and b
</table>

In [9]:
def known(words):
    #Return the subset of words that are actually in the dictionary."
    return {w for w in words if w in COUNTS}

def edits0(word): 
    return {word}

def edits2(word):
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

In [10]:
def edits1(word):
    pairs      = splits(word)
    deletes    = [a+b[1:]           for (a, b) in pairs if b]
    transposes = [a+b[1]+b[0]+b[2:] for (a, b) in pairs if len(b) > 1]
    replaces   = [a+c+b[1:]         for (a, b) in pairs for c in alphabet if b]
    inserts    = [a+c+b             for (a, b) in pairs for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

def splits(word):
    return [(word[:i], word[i:]) for i in range(len(word)+1)]

alphabet = 'abcdefghijklmnopqrstuvwxyz'

In [11]:
splits('wird')

[('', 'wird'), ('w', 'ird'), ('wi', 'rd'), ('wir', 'd'), ('wird', '')]

In [12]:
print(edits0('wird'))

{'wird'}


In [13]:
print(edits1('wird'))

{'cwird', 'lwird', 'wirdf', 'wikd', 'wyird', 'wirdq', 'wpird', 'wirvd', 'wprd', 'wirj', 'wirg', 'wiurd', 'wicd', 'wnrd', 'wcrd', 'wirb', 'bwird', 'wixrd', 'bird', 'wuird', 'wgrd', 'wirn', 'wiyrd', 'wirdl', 'wirdw', 'wiro', 'wirt', 'wizrd', 'wirp', 'wirkd', 'kird', 'wirq', 'wirdu', 'cird', 'wzird', 'wxrd', 'wiid', 'wimrd', 'wirfd', 'wirnd', 'pwird', 'wfrd', 'ewird', 'wirdm', 'wiird', 'wjrd', 'ywird', 'wiud', 'waird', 'aird', 'wbrd', 'wire', 'wwrd', 'wigrd', 'xird', 'wilrd', 'wirdx', 'qwird', 'winrd', 'widr', 'wisrd', 'wkird', 'wir', 'wurd', 'wiru', 'fird', 'wipd', 'dwird', 'wiprd', 'wirud', 'wwird', 'wizd', 'wirde', 'wfird', 'wrrd', 'nwird', 'whrd', 'wirdy', 'wild', 'oird', 'wird', 'wiord', 'wirdj', 'wirdn', 'wirk', 'wirdp', 'wirld', 'tird', 'owird', 'xwird', 'wbird', 'wqrd', 'wirds', 'wirdt', 'wirhd', 'sird', 'wmrd', 'wijd', 'qird', 'wivd', 'wirdb', 'wirz', 'wcird', 'wied', 'wirv', 'wibrd', 'wind', 'fwird', 'wifrd', 'wiryd', 'wiqd', 'gwird', 'witrd', 'yird', 'wvrd', 'wigd', 'hird', 'gi

In [14]:
print(len(edits2('wird')))

24254


In [15]:
list(map(correct, tokens('Speling errurs in somethink. Whutever; unusuel misteakes everyware?')))

['spelling',
 'errors',
 'in',
 'something',
 'whatever',
 'unusual',
 'mistakes',
 'everywhere']

Can we make the output prettier than that?

In [16]:
def correct_text(text):
    return re.sub('[a-zA-Z]+', correct_match, text)

def correct_match(match):
    word = match.group()
    return case_of(word)(correct(word.lower()))

def case_of(text):
    return (str.upper if text.isupper() else
            str.lower if text.islower() else
            str.title if text.istitle() else
            str)

In [17]:
correct_text('Speling Errurs IN somethink. Whutever; unusuel misteakes?')

'Spelling Errors IN something. Whatever; unusual mistakes?'

In [18]:
correct_text('Audiance sayzs: tumblr ...')

'Audience says: tumbler ...'

In [19]:
def pdist(counter):
    "Make a probability distribution, given evidence from a Counter."
    N = sum(list(counter.values()))
    return lambda x: counter[x]/N

P = pdist(COUNTS)

In [20]:
for w in tokens('"The" is most common word in English'):
    print(P(w), w)

0.0724066643445 the
0.00884296810325 is
0.000821507574969 most
0.00025966153526 common
0.000269613719538 word
0.0199496057578 in
0.000190900989338 english


In [21]:
def Pwords(words):
    return product(P(w) for w in words)

def product(nums):
    result = 1
    for x in nums:
        result *= x
    return result

In [22]:
tests = ['this is a test', 
         'this is a unusual test',
         'this is a neverbeforeseen test']

for test in tests:
    print(Pwords(tokens(test)), test)

2.9833963328e-11 this is a test
8.63747202302e-16 this is a unusual test
0.0 this is a neverbeforeseen test


## Predict test words

In [23]:
f = open('test.csv','r')

for line in f:
    row  = csv.reader([line])
    row  = list(row)[0]
    print(row[0]+','+row[1]+','+correct_text(row[1]))

ID,WRONG,WRONG
0,contenpted,contented
1,begining,beginning
2,problam,problem
3,dirven,driven
4,exstacy,ecstasy
5,guic,guns
6,localy,local
7,compair,company
8,pronounciation,pronunciation
9,transportibility,transportibility
10,miniscule,miniscule
11,independant,independent
12,aranged,arranged
13,poartry,party
14,leval,level
15,basicaly,basically
16,triangulaur,triangular
17,unexpcted,unexpected
18,stanerdizing,stanerdizing
19,varable,variable
20,futher,father
21,monitering,monitoring
22,biscits,biscuits
23,avaible,available
24,seperate,separate
25,neccesary,necessary
26,defenition,definition
27,receit,recent
28,remine,remind
29,inetials,initials
30,magnificnet,magnificent
31,annt,anna
32,intial,initial
33,ther,the
34,experances,experiences
35,biult,built
36,totaly,total
37,undersand,understand
38,southen,southern
39,definately,definitely
40,fisited,visited
41,volantry,voluntary
42,ment,men
43,recieve,receive
44,sorces,forces
45,wether,whether
46,usefull,useful
47,litriture,literature
48

396,oppinion,opinion
397,advice,advice
398,carear,career
399,resoved,removed
400,accessability,accessability
401,negligable,negligible
402,studens,student
403,faverable,favorable
404,tecniques,technique
405,atmospher,atmosphere
406,exsactly,exactly
407,especaily,especially
408,proprtions,proportions
409,goegraphicaly,geographical
410,encompasing,encompasing
411,stoped,stopped
412,relavent,relevant
413,thay,that
414,upplied,applied
415,elimiated,eliminated
416,containg,contains
417,conditining,conditining
418,sucssuful,sucssuful
419,quies,quiet
420,agravating,aggravating
421,errounous,erroneous
422,wepons,weapons
423,corparate,corporate
424,inconcievable,inconceivable
425,oranised,organised
426,sensable,sensible
427,studing,studying
428,eximination,examination
429,analiss,analysis
430,lonley,lonely
431,intrest,interest
432,comittees,committees
433,explaning,explaining
434,humor,humor
435,thorts,thorns
436,timeing,timing
437,dificulty,difficulty
438,currers,current
439,rgister,register
4

limit_output extension: Maximum message size of 10000 exceeded with 10096 characters

## Finding anamolies

In [138]:
f = open('test.csv','r')
g = open('test_submit.csv','r')

In [139]:
print('\nWords not present in Big.txt\n')
count = 1
for l1,l2 in zip(f,g):
    r1 = list(csv.reader([l1]))[0]
    r2 = list(csv.reader([l2]))[0]
    if r1[1] == r2[1]:
        for k,v in spell_errors.items():
            if r1[1] in v:
                print(str(count)+'. '+r1[1]+'\t'+k)
                break
        #print(str(count)+'. '+r1[1])
        count += 1


Words not present in Big.txt

1. transportibility	transportability
2. miniscule	minuscule
3. stanerdizing	standardizing
4. diagrammaticaally	diagrammatically
5. hierachial	hierarchical
6. planed	planted
7. addresable	addressable
8. hierachial	hierarchical
9. adabtable	adaptable
10. latter	later
11. graphicaly	graphically
12. thermawere	thermawear
13. orentated	orientated
14. unresloved	unresolved
15. unequaled	unequalled
16. avaiblity	availability
17. subtrcat	subtract
18. imidatly	immediately
19. exponentualy	exponentially
20. nessisitates	necessitates
21. unessasarily	unnecessarily
22. unavailble	unavailable
23. equaled	equalled
24. preffeson	profession
25. embelishing	embellishing
26. similar	similarity
27. forth	so_forth
28. pere	bad-tempered
29. necasery	necessary
30. unessessay	unnecessary
31. disaggreagte	disaggregate
32. et	Connecticut
33. chose	choose
34. advice	advised
35. accessability	accessibility
36. encompasing	encompassing
37. conditining	conditioning
38. sucssuful	suc

In [131]:
g = open('test_submit.csv','r')
count = 1
for line in g:
    row  = list(csv.reader([line]))[0]
    if correct_text(row[1]) not in COUNTS:
        for k,v in spell_errors.items():
            if row[1] in v:
                print(str(count)+'. '+row[1]+'\t'+k)
                break
        #print(str(count)+'. '+row[1])
        count += 1

2. transportibility	transportability
3. miniscule	minuscule
4. stanerdizing	standardizing
5. diagrammaticaally	diagrammatically
6. hierachial	hierarchical
7. addresable	addressable
8. hierachial	hierarchical
9. adabtable	adaptable
10. graphicaly	graphically
11. thermawere	thermawear
12. orentated	orientated
13. unresloved	unresolved
14. avaiblity	availability
15. subtrcat	subtract
16. imidatly	immediately
17. exponentualy	exponentially
18. nessisitates	necessitates
19. unessasarily	unnecessarily
20. unavailble	unavailable
21. preffeson	profession
22. embelishing	embellishing
23. necasery	necessary
24. unessessay	unnecessary
25. disaggreagte	disaggregate
26. accessability	accessibility
27. encompasing	encompassing
28. conditining	conditioning
29. sucssuful	successful
30. perametres	parameters
31. dissapoiting	disappointing
32. interogationg	interrogating
33. drasticaly	drastically
34. ineffiect	inefficient
35. citisum	criticism


In [24]:
spell_errors = {}
f = open('spell-errors.txt','r')

for line in f:
    sp = line.strip().replace('\n','').split(':')
    spell_errors[sp[0]] = sp[1]

## Upload

In [29]:
correction = ''
threshold  = 0.7
temp = 0.7

f = open('test.csv','r')

for line in f:
    row  = list(csv.reader([line]))[0]
    correction = correct_text(row[1])
    
    if row[1] == correction:
        for k,v in spell_errors.items():
            if row[1] in tokens(v):
                r = similar(row[1],k)
                if r >= threshold and r >= temp:
                    temp = r
                    correction = k.lower()
                    #break
            temp = 0.7
    if correction not in COUNTS:
        for k,v in spell_errors.items():
            if row[1] in tokens(v):
                r = similar(row[1],k)
                if r >= threshold and r >= temp:
                    temp = r
                    correction = k.lower()
                    #break
            temp = 0.7
    if similar(row[1],correction) < 0.90:
        for k,v in spell_errors.items():
            if row[1] in tokens(v):
                if correction != k:
                    r = similar(row[1],k)
                    if r >= threshold and r >= temp:
                        temp = r
                        correction = k.lower()
                        #break
            temp = 0.7
    print(row[0]+','+correction)

ID,WRONG
0,contented
1,beginning
2,problem
3,driven
4,ecstasy
5,guns
6,local
7,compare
8,pronunciation
9,transportability
10,minuscule
11,independent
12,arranged
13,poetry
14,level
15,basically
16,triangular
17,unexpected
18,standardizing
19,variable
20,further
21,monitoring
22,biscuits
23,available
24,separate
25,necessary
26,definition
27,receipt
28,remind
29,initials
30,magnificent
31,aunt
32,initial
33,they
34,experiences
35,built
36,total
37,understand
38,southern
39,definitely
40,visited
41,voluntary
42,meant
43,receive
44,sources
45,whether
46,useful
47,literature
48,valuable
49,delicate
50,clerical
51,splendid
52,between
53,completely
54,count
55,cemetery
56,special
57,latest
58,perhaps
59,remember
60,chapter
61,cake
62,various
63,february
64,pretend
65,choosing
66,rota
67,particular
68,awful
69,arrangement
70,challenges
71,laugh
72,often
73,someone
74,personnel
75,unique
76,diagrammatically
77,description
78,poems
79,purple
80,decide
81,articles
82,position
83,extended
84,hier

References:

https://web.stanford.edu/class/cs276/pa/pa2.p

http://web.stanford.edu/~jurafsky/slp3/5.pdf

http://norvig.com/ngrams/spell-errors.txt

http://nbviewer.jupyter.org/url/norvig.com/ipython/How%20to%20Do%20Things%20with%20Words.ipynb

http://norvig.com/big.txt

http://norvig.com/ngrams/count_1w.txt

http://norvig.com/ngrams/